# Lab 2: Classification
## AirBnB Listings for Los Angeles Area
### Jason McDonald, Miguel Bonilla, Zachary Bunn

**Table of Contents**

- [Data Preparation: Part 1](#Data-Preparation:-Part-1)
- [Data Preparation: Part 2](#Data-Preparation:-Part-2)
- [Modeling and Evaulation 1](#Modeling-and-Evaluation-1)
- [Modeling and Evaulation 2](#Modeling-and-Evaluation-2)
- [Modeling and Evaulation 3](#Modeling-and-Evaluation-3)
- [Modeling and Evaulation 4](#Modeling-and-Evaluation-4)
- [Modeling and Evaulation 5](#Modeling-and-Evaluation-5)
- [Modeling and Evaulation 6](#Modeling-and-Evaluation-6)
- [Deployment](#Deployment)
- [Exceptional Work](#Exceptional-Work)

### Data Preparation: Part 1

## Data Preparation: Part 2

## Modeling and Evaluation 1

## Modeling and Evaluation 2

## Modeling and Evaluation 3

## Modeling and Evaluation 4

## Modeling and Evaluation 5

## Modeling and Evaluation 6

## Deployment

## Exceptional Work